In [ ]:
!python --version

Python 3.10.12


In [27]:
!git add .
!git commit -m "abc"
!git push origin main

[main 6689aa8] abc
 18 files changed, 282 insertions(+), 10 deletions(-)
 create mode 100644 __pycache__/final_torch_model.cpython-310.pyc
 create mode 100644 __pycache__/myNet.cpython-310.pyc
 create mode 100644 __pycache__/optimize.cpython-310.pyc
 create mode 100644 __pycache__/policy.cpython-310.pyc
 create mode 100644 __pycache__/replayBuffer.cpython-310.pyc
 create mode 100644 __pycache__/torch_model.cpython-310.pyc
 create mode 100644 __pycache__/train.cpython-310.pyc
 create mode 100644 models/blue_8.pt
 copy models/{blue_dqn_final.pt => blue_dqn_fina2l.pt} (100%)
 rewrite models/blue_dqn_final.pt (95%)
 create mode 100644 optimize.py
 create mode 100644 policy.py
 create mode 100644 replayBuffer.py
 create mode 100644 video/new_pretrained.mp4
 create mode 100644 video/pretrained.mp4
 create mode 100644 video/random.mp4
Enumerating objects: 26, done.
Counting objects: 100% (26/26), done.
Delta compression using up to 2 threads
Compressing objects: 100% (21/21), done.
Writing ob

In [2]:
%%capture
!git clone https://github.com/phamtiep/RL-final

%cd RL-final

In [29]:
!git add .
!git commit -m 'update'
!git push

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


In [2]:
!pip install git+https://github.com/Farama-Foundation/MAgent2

  Cloning https://github.com/Farama-Foundation/MAgent2 to /tmp/pip-req-build-sxgpkk7x
  Running command git clone --filter=blob:none --quiet https://github.com/Farama-Foundation/MAgent2 /tmp/pip-req-build-sxgpkk7x
  Resolved https://github.com/Farama-Foundation/MAgent2 to commit b2ddd49445368cf85d4d4e1edcddae2e28aa1406
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from collections import defaultdict, deque
import random
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from magent2.environments import battle_v4

In [4]:
from replayBuffer import MultiAgentReplayBuffer
from optimize import *
from myNet import QNetwork
from policy import epsilon_greedy_policy
from train import train_agents


In [5]:
env = battle_v4.env(map_size=45, minimap_mode=False, step_reward=0.01,
                        dead_penalty=-2, attack_penalty=-0.1, attack_opponent_reward=2,
                        max_cycles=300, extra_features=False, render_mode="rgb_array")

BATCH_SIZE = 128
GAMMA = 0.9
EPS_START = 1
EPS_END = 0.1
EPS_DECAY = 50
TAU = 0.005
LR = 1e-4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

observation_shape = env.observation_space("blue_0").shape
action_shape = env.action_space("blue_0").n

policy_net = QNetwork(observation_shape, action_shape).to(device)
red_policy_net = QNetwork(observation_shape, action_shape).to(device)
target_net = QNetwork(observation_shape, action_shape).to(device)
target_net.load_state_dict(policy_net.state_dict())
optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)

pretrained_net = QNetwork(observation_shape, action_shape).to(device)
pretrained_net.load_state_dict(torch.load("models/red.pt", map_location=device, weights_only=True))

try:
    checkpoint = torch.load("models/blue_2.pt", map_location=device, weights_only=True)
    policy_net.load_state_dict(checkpoint["policy_net_state_dict"])
    target_net.load_state_dict(checkpoint["target_net_state_dict"])
    red_policy_net.load_state_dict(checkpoint["policy_net_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    episode = checkpoint["episode"]
    print(f"Start with episode: {episode}")
except Exception as e:
    print(f"No model found!")
    episode = 0

buffer = MultiAgentReplayBuffer(10000, observation_shape, action_shape)
steps_done = episode
episode_rewards = []
episode_losses = []
running_loss = 0.0
num_episodes = 60

cuda
No model found!


In [13]:
train_agents(env, num_episodes, policy_net, target_net, red_policy_net, buffer, optimizer, steps_done, episode, pretrained_net,TAU, EPS_DECAY= EPS_DECAY, EPS_END = EPS_END, EPS_START = EPS_START, device = device, BATCH_SIZE = BATCH_SIZE, GAMMA = GAMMA)

KeyboardInterrupt: 

In [24]:
checkpoint = torch.load("models/blue_8.pt", map_location=device, weights_only=True)
a = checkpoint["policy_net_state_dict"]
torch.save(a, 'models/blue_dqn_final.pt')

In [26]:
!python demo.py

Done recording random agents
Done recording pretrained agents
Done recording new pretrained agents
